<a href="https://colab.research.google.com/github/shernee/06_CMPE297/blob/main/Function_Calling_OpenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 2.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [ ]:
import openai
import requests

In [ ]:
openai.api_key = ""

In [ ]:
model = "gpt-3.5-turbo-0613"

In [ ]:
def chat_completion_request(messages, functions, model=model):
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + openai.api_key,
    }

    json_data = {"model": model, "messages": messages}
    if functions is not None:
        json_data.update({"functions": functions})

    try:
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=json_data,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

In [ ]:
functions = [
    {
        "name": "get_avg_market_price",
        "description": "Get the market price of a car",
        "parameters": {
            "type": "object",
            "properties": {
                "model": {
                    "type": "string",
                    "description": "The make and model, e.g. Nissan Altima",
                },
                "year": {
                    "type": "string",
                    "description": "Year of manufacture, e.g. 2020",
                },
            },
            "required": ["model", "year"],
        },
    },
    {
        "name": "get_market_price_for_location",
        "description": "Get an the market price of a vehicle in a particular location",
        "parameters": {
            "type": "object",
            "properties": {
                "model": {
                    "type": "string",
                    "description": "The make and model, e.g. Nissan Altima",
                },
                "year": {
                    "type": "string",
                    "description": "Year of manufacture, e.g. 2020",
                },
                "location": {
                    "type": "string",
                    "description": "State and City of the search, e.g. Des Moines, IA"
                }
            },
            "required": ["model", "year", "location"]
        },
    },
]

In [ ]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for missing function property values if they are required."})
messages.append({"role": "user", "content": "What's the market price of a Toyota Prius"})
chat_response = chat_completion_request(
    messages, functions=functions
)
assistant_message = chat_response.json()["choices"][0]["message"]
messages.append(assistant_message)
assistant_message

{'role': 'assistant',
 'content': None,
 'function_call': {'name': 'get_avg_market_price',
  'arguments': '{\n  "model": "Toyota Prius",\n  "year": "2021"\n}'}}

In [ ]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for missing function property values if they are required."})
messages.append({"role": "user", "content": "what is the price of a Toyota Prius in location x"})
chat_response = chat_completion_request(
    messages, functions=functions
)
assistant_message = chat_response.json()["choices"][0]["message"]
messages.append(assistant_message)
assistant_message

{'role': 'assistant',
 'content': None,
 'function_call': {'name': 'get_market_price_for_location',
  'arguments': '{\n  "model": "Toyota Prius",\n  "year": "2021",\n  "location": "location x"\n}'}}